In [1]:
import os
import numpy as np # to process the data
import random
from PIL import Image
from sklearn.model_selection import train_test_split

In [44]:
class FlowerDataset:
    def __init__(self, directory):
        self.directory = directory
        self.images = []
        self.labels = []

    def load_data(self):
        for i, img in enumerate(os.listdir(self.directory)):
            if i >= 10:  # only load the first 10 images
                break
            try:
                img_array = Image.open(os.path.join(self.directory, img))  # read the image
                rgb_array = img_array.convert('RGB')  # ensure image is RGB
                resized_array = rgb_array.resize((150, 150))  # resize the image
                self.images.append(np.array(resized_array))
                label = img.split('_')[0]  # extract label from filename
                self.labels.append(label)
                # print(f"Loaded image {img} with label {label}")
            except Exception as e:
                print(f"Error loading image {img}: {e}")
            
    def preprocess_data(self):
        self.images = np.array(self.images).reshape(-1, 150, 150, 3)
        self.images = self.images / 255.0  # normalize pixel values
        self.labels = np.array(self.labels)
        print(f"Loaded {len(self.images)} images")


# Load and preprocess the data
dataset = FlowerDataset('flowers')
dataset.load_data()
dataset.preprocess_data()

print(f"Shape of the preprocessed images: {dataset.images.shape}")
# Print the dimensions of the first image
print(f"Dimensions of the first preprocessed image: {dataset.images[0].shape}")

# Get the images and labels
images = dataset.images
labels = dataset.labels

# Split the data into training, validation, and test sets
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

print(f"Train set size: {len(x_train)}, Validation set size: {len(x_val)}, Test set size: {len(x_test)}")

Loaded 10 images
Shape of the preprocessed images: (10, 150, 150, 3)
Dimensions of the first preprocessed image: (150, 150, 3)
Train set size: 7, Validation set size: 1, Test set size: 2


In [3]:
class Computing:
    # ReLU activation function
    def relu(self, x):
        return max(0, x)

    # Derivative of ReLU
    def relu_derivative(self, x):
        return 1 if x > 0 else 0

    # Softmax function
    def softmax(self, x):
        e_x = [self.exp(i) for i in x]
        return [i / sum(e_x) for i in e_x]

    # Exponential function
    def exp(self, x):
        return 2.718281828459045 ** x  # approximation of e^x

    # Cross-entropy loss function
    def cross_entropy(self, predictions, targets):
        return -sum([targets[i]*self.log(predictions[i]) for i in range(len(predictions))])

    # Natural logarithm function
    def log(self, x):
        # Implement a method to compute the natural logarithm of x
        pass

In [37]:
class ConvLayer:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = [[[[random.random() for _ in range(3)] for _ in range(filter_size)] for _ in range(filter_size)] for _ in range(num_filters)]
        self.computing = Computing()

    def zeros(shape):
        if len(shape) == 1:
            return [0 for _ in range(shape[0])]
        elif len(shape) == 2:
            return [[0 for _ in range(shape[1])] for _ in range(shape[0])]
        elif len(shape) == 3:
            return [[[0 for _ in range(shape[2])] for _ in range(shape[1])] for _ in range(shape[0])]

    # Forward pass
    def forward(self, input):
        h, w, num_filters = len(input), len(input[0]), self.num_filters
        output = zeros((h - self.filter_size + 1, w - self.filter_size + 1, num_filters))
        for i in range(h - self.filter_size + 1):
            for j in range(w - self.filter_size + 1):
                for f in range(num_filters):
                    output[i][j][f] = dot(input[i:i+self.filter_size, j:j+self.filter_size], self.filters[f])
        return output

    # Backward pass
    def backward(self, input, grad_output):
        grad_input = zeros(input.shape)
        grad_filters = zeros(self.filters.shape)
        for i in range(input.shape[0] - self.filter_size + 1):
            for j in range(input.shape[1] - self.filter_size + 1):
                for f in range(self.num_filters):
                    grad_input[i:i+self.filter_size, j:j+self.filter_size] += grad_output[i, j, f] * self.filters[f]
                    grad_filters[f] += grad_output[i, j, f] * input[i:i+self.filter_size, j:j+self.filter_size]
        self.filters -= self.learning_rate * grad_filters
        return grad_input


class MaxPool2:
    def __init__(self, filter_size):
        self.filter_size = filter_size

    # Forward pass
    def forward(self, input):
        self.last_input = input

        # Get the dimensions of the input
        batch_size, h, w, num_filters = len(input), len(input[0]), len(input[0][0]), len(input[0][0][0])
        output = [[[[0 for _ in range(num_filters)] for _ in range(w // self.filter_size)] for _ in range(h // self.filter_size)] for _ in range(batch_size)]

        for b in range(batch_size):
            for i in range(len(output[0])):
                for j in range(len(output[0][0])):
                    for f in range(num_filters):
                        patch = []
                        for di in range(self.filter_size):
                            for dj in range(self.filter_size):
                                patch.append(input[b][i* self.filter_size + di][j* self.filter_size + dj][f])
                        output[b][i][j][f] = max(patch)

        return output

    # Backward pass
    def backward(self, input, grad_output):
        grad_input = np.zeros(input.shape)

        for i in range(input.shape[0] // self.filter_size):
            for j in range(input.shape[1] // self.filter_size):
                h_start = i * self.filter_size
                h_end = (i + 1) * self.filter_size
                w_start = j * self.filter_size
                w_end = (j + 1) * self.filter_size

                area_grad = grad_output[i, j]
                max_val = np.amax(input[h_start:h_end, w_start:w_end], axis=(0, 1))
                mask = input[h_start:h_end, w_start:w_end] == max_val
                grad_input[h_start:h_end, w_start:w_end] = mask * area_grad

        return grad_input


class Flatten:
    # Forward pass
    def forward(self, input):
        self.last_input_shape = input.shape
        return input.flatten()

    # Backward pass
    def backward(self, input, grad_output):
       return grad_output.reshape(self.last_input_shape)


class Dropout:
    def __init__(self, rate):
        self.rate = rate

    # Forward pass
    def forward(self, input):
        self.mask = (np.random.rand(*input.shape) > self.rate) / (1.0 - self.rate)
        return input * self.mask

    # Backward pass
    def backward(self, input, grad_output):
        return grad_output * self.mask


class Dense:
    def __init__(self, input_units, output_units):
        self.weights = [[0.01 for _ in range(output_units)] for _ in range(input_units)]
        self.biases = [0 for _ in range(output_units)]
        self.computing = Computing()

    # Forward pass
    def forward(self, input):
        output = [0 for _ in range(len(self.biases))]
        for i in range(len(input)):
            for j in range(len(self.weights[0])):
                output[j] += input[i] * self.weights[i][j]
        for i in range(len(self.biases)):
            output[i] += self.biases[i]
        return output

    # Backward pass
    def backward(self, input, grad_output):
        grad_input = np.dot(grad_output, self.weights.T)
        grad_weights = np.dot(input.T, grad_output)
        grad_biases = grad_output.mean(axis=0)*input.shape[0]

        assert grad_weights.shape == self.weights.shape and grad_biases.shape == self.biases.shape
        self.weights = self.weights - learning_rate * grad_weights
        self.biases = self.biases - learning_rate * grad_biases

        return grad_input


class Network:
    def __init__(self):
        self.layers = []
        self.computing = Computing()

    # Add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # Forward pass
    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    # Compute loss
    def compute_loss(self, predictions, labels):
        return self.computing.cross_entropy(predictions, labels)

    # Compute accuracy
    def compute_accuracy(self, predictions, labels):
        correct_predictions = np.sum(np.argmax(predictions, axis=1) == np.argmax(labels, axis=1))
        total_predictions = predictions.shape[0]
        return correct_predictions / total_predictions

    # Backward pass
    def backward(self, X, y):
        # Compute the gradient of the loss
        grad_output = self.compute_loss_grad(self.layers[-1].output, y)
        # Propagate the gradient through the layers
        for layer in reversed(self.layers):
            grad_output = layer.backward(layer.last_input, grad_output)


In [38]:
def custom_cnn():
    # Initialize the network
    network = Network()

    # Add layers to the network
    network.add(ConvLayer(32, 3))  # Convolutional layer with 32 filters of size 3x3
    network.add(MaxPool2(2))  # Max pooling layer with filter size 2x2
    network.add(ConvLayer(64, 3))  # Another convolutional layer
    network.add(MaxPool2(2))  # Another max pooling layer
    network.add(ConvLayer(128, 3))  # Another convolutional layer
    network.add(MaxPool2(2))  # Another max pooling layer
    network.add(ConvLayer(256, 3))  # Another convolutional layer
    network.add(MaxPool2(2))  # Another max pooling layer
    network.add(Flatten())  # Flatten layer to prepare for fully connected layers
    network.add(Dense(1024, 512))  # Fully connected layer with 1024 input units and 512 output units
    network.add(Dense(512, 256))  # Fully connected layer with 512 input units and 256 output units
    network.add(Dense(256, 128))  # Fully connected layer with 256 input units and 128 output units
    network.add(Dense(128, 10))  # Output layer with 10 units (one for each class)

    return network

In [39]:
def train_config(network, learning_rate, activator, loss, threshold, early_stopping):
    # Set the network's activator, loss, and learning rate
    network.activator = activator
    network.loss = loss
    network.learning_rate = learning_rate
    network.threshold = threshold
    network.early_stopping = early_stopping

In [40]:
def train(network, x_train, y_train, epochs):
    # Initialize variables for early stopping
    best_loss = float('inf')
    epochs_without_improvement = 0

    # Loop over the epochs
    for epoch in range(epochs):
        # Perform a forward pass and compute the loss
        predictions = network.forward(x_train)
        loss = network.compute_loss(predictions, y_train)

        # Compute the accuracy
        accuracy = network.compute_accuracy(predictions, y_train)

        # Perform a backward pass and update the weights
        network.backward(x_train, y_train)

        # Print the loss and accuracy for this epoch
        print(f'Epoch {epoch+1}/{epochs} - loss: {loss}, accuracy: {accuracy}')

        # Check for early stopping
        if loss < best_loss - network.threshold:
            best_loss = loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement == network.early_stopping:
                print('Early stopping triggered')
                break

In [41]:
# Create the network
network = custom_cnn()
print(network)

In [42]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"  # for all three GPUs
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # for the first GPU
# import gpustat
# print(gpustat.print_gpustat())
print(os.getpid())

3977641


In [43]:
# Define training parameters
epochs = 1
learning_rate = 1
activator = 'relu'
loss = 'cross_entropy'
threshold = 0.01
early_stopping = 10

# Configure the network for training
train_config(network, learning_rate, activator, loss, threshold, early_stopping)

# Train the network
train(network, x_train, y_train, epochs)
# TOOOOOOOOOOOOOOOOOOOOO SLOOOOOOOOOOOWWWWWWWWWWWWWWWWWWWWWW

NameError: name 'zeros' is not defined